# Forecasting

The following sections detail the econometric and machine learning pipeline used to evaluate the predictive utility of the INSEI sentiment index against the official Consumer Price Index (CPI). 

The analysis employs a "Control vs. Test" experimental design, comparing baseline models (using only historical inflation data) against hybrid models (using historical inflation + news sentiment) across three distinct architectures: Ridge Regression, Vector Autoregression (VAR), and Long Short-Term Memory (LSTM).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os

from statsmodels.tsa.api import VAR
from statsmodels.tsa.ar_model import AutoReg
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.random.set_seed(42)
np.random.seed(42)
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
def calculate_metrics(y_true, y_pred, model_name="Model"):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    print(f"--- {model_name} ---")
    print(f"MAE:  {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R2:   {r2:.4f}")
    print("-" * 30)
    
    return {"Model": model_name, "MAE": mae, "RMSE": rmse, "R2": r2}

results_summary = []

# Data Load & Split

In [2]:
# Load the Aligned Data (Created in File 8)
df = pd.read_csv("insei_all_model_raw_granger_fullmodif.csv")
df["year_month_dt"] = pd.to_datetime(df["year_month_dt"])
df = df.set_index("year_month_dt")

# Define Columns
TARGET_COL = "Rate"
SENTIMENT_COLS = ["insei_indobert_raw", "insei_indolem_raw", "insei_mdeberta_raw","insei_gemini_raw"]

# Split Train/Test
CUTOFF_DATE = "2024-01-01"
train_df = df[df.index < CUTOFF_DATE].copy()
test_df = df[df.index >= CUTOFF_DATE].copy()

print(f"Training Samples: {len(train_df)} (Jan 2017 - Dec 2023)")
print(f"Testing Samples:  {len(test_df)}  (Jan 2024 - Oct 2025)")

Training Samples: 78 (Jan 2017 - Dec 2023)
Testing Samples:  22  (Jan 2024 - Oct 2025)


In [3]:
df.head()

,year_month,insei_indobert_raw,insei_indolem_raw,insei_mdeberta_raw,insei_gemini_raw,insei_indobert_norm,insei_indobert_smooth,insei_indobert_lagged,insei_indobert_final,insei_indolem_norm,...,insei_gemini_norm,insei_gemini_smooth,insei_gemini_lagged,insei_gemini_final,insei_indobert_lagged_raw,insei_indolem_lagged_raw,insei_mdeberta_lagged_raw,insei_gemini_lagged_raw,Rate,year_month_str
year_month_dt,,,,,,,,,,,,,,,,,,,,,
2017-07-01,2017-07,-0.078912,0.011876,-0.050584,0.048780,0.00000,0.000000,0.0,50.0,0.000000,...,0.000000,0.000000,0.0,50.0,0.018024,0.264608,0.222023,0.230769,0.22,2017-07
2017-08-01,2017-08,-0.052705,0.018444,-0.051758,0.040000,0.00000,0.000000,0.0,50.0,0.000000,...,0.000000,0.000000,0.0,50.0,0.098984,0.229115,0.142892,0.296296,-0.07,2017-08
2017-09-01,2017-09,0.044486,0.115167,0.095657,0.250000,0.00000,0.000000,0.0,50.0,0.000000,...,0.000000,0.000000,0.0,50.0,-0.078912,0.011876,-0.050584,0.048780,0.13,2017-09
2017-10-01,2017-10,-0.026432,0.043418,0.026158,0.081633,-0.40897,-0.136323,0.0,50.0,-0.639696,...,-0.671247,-0.223749,0.0,50.0,-0.052705,0.018444,-0.051758,0.040000,0.01,2017-10
2017-11-01,2017-11,-0.124479,0.053617,-0.018199,0.163265,-1.39928,-0.602750,0.0,50.0,-0.500887,...,0.044214,-0.209011,0.0,50.0,0.044486,0.115167,0.095657,0.250000,0.20,2017-11


Loads the aligned dataset insei_all_model_cpi_aligned_lag2.csv, which contains monthly observations of the CPI Inflation Rate and the corresponding 2-month lagged sentiment indices from four models (IndoBERT, IndoLEM, mDeBERTa, Gemini). 

The date column is converted to a datetime object and set as the index. A  temporal split is defined using CUTOFF_DATE = "2024-01-01".
- Training Set (2017–2023): 82 months (Jan 2017 – Dec 2023) period covers pre-pandemic stability, the COVID-19 shock, and the initial recovery. Used to fit model parameters.
- Testing Set (2024–2025): This forecast horizon tests the models' ability to generalize to the most recent economic conditions. Used solely for out-of-sample evaluation.


# Var

In [4]:
# Define a separate function just for the Baseline
def run_baseline_ar(train, test):
    ar_model = AutoReg(train[TARGET_COL].dropna(), lags=12, old_names=False).fit()
    
    pred_start = len(train)
    pred_end = len(train) + len(test) - 1
    preds_base = ar_model.predict(start=pred_start, end=pred_end, dynamic=False).values
    
    test_clean = test[TARGET_COL].iloc[-len(preds_base):]
    # Calculate and PRINT metrics only once
    return calculate_metrics(test_clean, preds_base, "VAR_Baseline_AR_Only")

# Modify comparison function to ACCEPT the baseline metrics
def run_var_hybrid_only(train, test, sentiment_col, baseline_metrics):
    #  SCENARIO B: Hybrid (Bi-variate VAR) 
    cols = [TARGET_COL, sentiment_col]
    train_var = train[cols].dropna()
    test_var = test[cols].dropna()
    
    model = VAR(train_var)
    model_fitted = model.fit(maxlags=12, ic='bic')    
    lag_order = model_fitted.k_ar
    
    history = train_var.values[-lag_order:].tolist()
    predictions = []
    
    for i in range(len(test_var)):
        input_data = np.array(history[-lag_order:])
        fc = model_fitted.forecast(y=input_data, steps=1)
        predictions.append(fc[0][0])
        history.append(test_var.iloc[i].values)
        
    metrics_hyb = calculate_metrics(test_var[TARGET_COL], predictions, f"VAR_Hybrid_{sentiment_col}")
    
    # Compare with the pre-calculated baseline
    imp = baseline_metrics['RMSE'] - metrics_hyb['RMSE']
    print(f"VAR Improvement with {sentiment_col}: {imp:.4f}")
    
    return metrics_hyb

baseline_metrics = run_baseline_ar(train_df, test_df)
results_summary.append(baseline_metrics)

for sent in SENTIMENT_COLS:
    hyb_metrics = run_var_hybrid_only(train_df, test_df, sent, baseline_metrics)
    results_summary.append(hyb_metrics)

--- VAR_Baseline_AR_Only ---
MAE:  0.3499
RMSE: 0.5002
R2:   -0.0017
------------------------------
--- VAR_Hybrid_insei_indobert_raw ---
MAE:  0.3561
RMSE: 0.5042
R2:   -0.0176
------------------------------
VAR Improvement with insei_indobert_raw: -0.0039
--- VAR_Hybrid_insei_indolem_raw ---
MAE:  0.3598
RMSE: 0.5086
R2:   -0.0355
------------------------------
VAR Improvement with insei_indolem_raw: -0.0084
--- VAR_Hybrid_insei_mdeberta_raw ---
MAE:  0.3545
RMSE: 0.5065
R2:   -0.0270
------------------------------
VAR Improvement with insei_mdeberta_raw: -0.0063
--- VAR_Hybrid_insei_gemini_raw ---
MAE:  0.3582
RMSE: 0.5096
R2:   -0.0395
------------------------------
VAR Improvement with insei_gemini_raw: -0.0093


c:\Users\Lenovo\.conda\envs\stable_diff_model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\Lenovo\.conda\envs\stable_diff_model\Lib\site-packages\statsmodels\tsa\deterministic.py:308: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and Index with a unit increment support extending. The index is set will contain the position relative to the data length.
  fcast_index = self._extend_index(index, steps, forecast_index)
c:\Users\Lenovo\.conda\envs\stable_diff_model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\Lenovo\.conda\envs\stable_diff_model\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred fr

VAR is a system of equations where every variable is "endogenous" (they all influence each other). This is crucial because Inflation affects Sentiment, and Sentiment affects Inflation.
- The System of Equations:Let $Y_t$ be a vector containing both variables: $Y_t = \begin{bmatrix} \text{Rate}_t \\ \text{Sentiment}_t \end{bmatrix}$.The VAR(p) model is:$$Y_t = c + A_1 Y_{t-1} + A_2 Y_{t-2} + \dots + A_p Y_{t-p} + e_t$$
Where $A_k$ are $2 \times 2$ coefficient matrices. Written out for Inflation ($y_t$):$$y_t = c_1 + \sum_{k=1}^{p} a_{11,k} y_{t-k} + \sum_{k=1}^{p} a_{12,k} S_{t-k} + e_{1,t}$$
- Unlike Ridge, VAR captures the feedback loop. It understands that a shock to Sentiment today might raise Inflation tomorrow, which in turn scares the public and lowers Sentiment further.

The experiment follows a rigorous Baseline vs. Hybrid design:
- Baseline (AutoReg): A univariate Autoregressive model is trained solely on the Rate column using AutoReg. It predicts the test set using only historical inflation inertia. This serves as the control group.
- Hybrid (Bi-variate VAR): A Vector Autoregression model is trained on both Rate and the specific sentiment_col. The model order (lag length) is automatically selected using the Bayesian Information Criterion (BIC) to prevent overfitting. The forecast is generated using a rolling window approach, where the model uses the actual observed history (teacher forcing) to predict one step ahead for the entire test period.

Baseline Performance: The AR model yields an RMSE of 0.5002.

Hybrid Performance: The hybrid models consistently underperfrom the baseline.

# Ridge

In [5]:
def create_lags(data, sent_col=None, lags=12):
    df_lag = pd.DataFrame()
    # A. Autoregressive Features
    for i in range(1, lags + 1):
        df_lag[f"Rate_L{i}"] = data[TARGET_COL].shift(i)
        
    # B. Sentiment Features (Only if sent_col is provided)
    if sent_col: 
        for i in range(1, lags + 1):
            df_lag[f"Sent_L{i}"] = data[sent_col].shift(i)
            
    df_lag["Target"] = data[TARGET_COL]
    return df_lag.dropna()

def run_ridge_baseline(train, test, lags=12):
    # Prepare Data (No sentiment col passed)
    train_feats = create_lags(train, sent_col=None, lags=lags)
    test_feats = create_lags(test, sent_col=None, lags=lags)
    
    X_train = train_feats.drop("Target", axis=1)
    y_train = train_feats["Target"]
    X_test = test_feats.drop("Target", axis=1)
    y_test = test_feats["Target"]
    
    # Scale
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)
    
    # Train & Predict
    model = RidgeCV(cv=5).fit(X_train_s, y_train)
    preds = model.predict(X_test_s)
    
    return calculate_metrics(y_test, preds, "Ridge_Baseline_AR_Only")

def run_ridge_hybrid(train, test, sentiment_col, baseline_metrics, lags=12):
    # Prepare Data (With Sentiment)
    train_feats = create_lags(train, sentiment_col, lags)
    test_feats = create_lags(test, sentiment_col, lags)
    
    X_train = train_feats.drop("Target", axis=1)
    y_train = train_feats["Target"]
    X_test = test_feats.drop("Target", axis=1)
    y_test = test_feats["Target"]
    
    # Scale
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)
    
    # Train & Predict
    model = RidgeCV(cv=5).fit(X_train_s, y_train)
    preds = model.predict(X_test_s)
    
    metrics_hyb = calculate_metrics(y_test, preds, f"Ridge_Hybrid_{sentiment_col}")
    
    # Compare
    imp = baseline_metrics['RMSE'] - metrics_hyb['RMSE']
    print(f"Ridge Improvement with {sentiment_col}: {imp:.4f}")
    
    return metrics_hyb

ridge_base_metrics = run_ridge_baseline(train_df, test_df)
results_summary.append(ridge_base_metrics)

for sent in SENTIMENT_COLS:
    hyb = run_ridge_hybrid(train_df, test_df, sent, ridge_base_metrics)
    results_summary.append(hyb)

--- Ridge_Baseline_AR_Only ---
MAE:  0.6197
RMSE: 0.7509
R2:   -0.1636
------------------------------
--- Ridge_Hybrid_insei_indobert_raw ---
MAE:  0.5982
RMSE: 0.7404
R2:   -0.1311
------------------------------
Ridge Improvement with insei_indobert_raw: 0.0105
--- Ridge_Hybrid_insei_indolem_raw ---
MAE:  0.5826
RMSE: 0.7299
R2:   -0.0993
------------------------------
Ridge Improvement with insei_indolem_raw: 0.0210
--- Ridge_Hybrid_insei_mdeberta_raw ---
MAE:  0.5939
RMSE: 0.7463
R2:   -0.1492
------------------------------
Ridge Improvement with insei_mdeberta_raw: 0.0046
--- Ridge_Hybrid_insei_gemini_raw ---
MAE:  0.6220
RMSE: 0.7728
R2:   -0.2325
------------------------------
Ridge Improvement with insei_gemini_raw: -0.0219


Ridge Regression (Linear Baseline vs. Hybrid)

Ridge is a regularized linear model. It works by shrinking coefficients ($\beta$) to prevent overfitting, which is critical when you have many correlated lags (e.g., Lag 1 is highly correlated with Lag 2).
- The Objective Function: Ridge minimizes the Sum of Squared Errors (SSE) plus a penalty term ($\lambda$) for the size of the coefficients:$$\hat{\beta} = \arg\min_{\beta} \left( \sum_{t=1}^{T} (y_t - \hat{y}_t)^2 + \lambda \sum_{j=1}^{p} \beta_j^2 \right)$$
- Scenario A: AR Baseline (Control). The model predicts inflation ($y_t$) using only its own past values ($y_{t-k}$):$$y_t = \alpha + \sum_{k=1}^{12} \beta_k y_{t-k} + \epsilon_t$$
- Scenario B: Hybrid Model (Test). The model includes the Sentiment Index ($S_{t-k}$) as an exogenous regressor:$$y_t = \alpha + \sum_{k=1}^{12} \beta_k y_{t-k} + \sum_{k=1}^{12} \gamma_k S_{t-k} + \epsilon_t$$

- If $\gamma_k \neq 0$ and the error decreases, sentiment provides linear information gain.

The create_lags function transforms the time-series problem into a supervised regression format. It generates 12 lags for the Inflation Rate ($Rate_{t-1} \dots Rate_{t-12}$) and, for the hybrid models, 12 lags for the Sentiment Index.
- Baseline: Uses only the 12 Rate lags as features.
- Hybrid: Uses 12 Rate lags + 12 Sentiment lags.
- Model: RidgeCV is used with 5-fold cross-validation to automatically select the optimal regularization strength ($\alpha$), effectively handling the high correlation between lagged features. Standard scaling is applied to all features to ensure the regularization penalty is applied uniformly.

The Ridge results highlight a significant performance gain:
- Baseline RMSE: 0.7509
- Hybrid (IndoLEM) RMSE: 0.7299
- Improvement: The inclusion of IndoLEM sentiment reduced the RMSE by 3%

The linear models benefited most from the sentiment data. The substantial drop in error suggests that news sentiment helps correct the linear projections of inflation, likely by capturing "shocks" that purely autoregressive features miss.

# LSTM

In [6]:
LOOKBACK = 12
EPOCHS = 50
BATCH_SIZE = 16

In [7]:
def create_sequences(data, lookback):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i+lookback])
        y.append(data[i+lookback, 0]) # 0 is always Rate
    return np.array(X), np.array(y)

# Generic training function
def train_predict_lstm(train, test, feature_cols, model_name):
    # Data Prep
    train_data = train[feature_cols].values
    test_data = test[feature_cols].values
    
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data)
    test_scaled = scaler.transform(test_data)
    
    X_train, y_train = create_sequences(train_scaled, LOOKBACK)
    n_features = len(feature_cols)
    
    # Model
    model = Sequential([
        LSTM(64, activation='relu', input_shape=(LOOKBACK, n_features), return_sequences=False),
        Dropout(0.1),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    
    early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0, callbacks=[early_stop])
    
    # Forecast Loop
    current_window = train_scaled[-LOOKBACK:].reshape(1, LOOKBACK, n_features)
    predictions = []
    
    for i in range(len(test_scaled)):
        pred_scaled = model.predict(current_window, verbose=0)[0, 0]
        predictions.append(pred_scaled)
        
        if n_features == 1:
            new_row = np.array([[pred_scaled]]) 
        else:
            next_sentiment = test_scaled[i, 1] 
            new_row = np.array([[pred_scaled, next_sentiment]])
            
        current_window = np.append(current_window[:, 1:, :], [new_row], axis=1)
        
    # Invert Scaling
    dummy = np.zeros((len(predictions), n_features))
    dummy[:, 0] = predictions
    preds_final = scaler.inverse_transform(dummy)[:, 0]
    
    return calculate_metrics(test[TARGET_COL].values, preds_final, model_name)

lstm_base_metrics = train_predict_lstm(train_df, test_df, [TARGET_COL], "LSTM_Baseline_AR_Only")
results_summary.append(lstm_base_metrics)

for sent in SENTIMENT_COLS:
    # We use [TARGET_COL, sent]
    hyb_metrics = train_predict_lstm(train_df, test_df, [TARGET_COL, sent], f"LSTM_Hybrid_{sent}")
    
    # Compare
    imp = lstm_base_metrics['RMSE'] - hyb_metrics['RMSE']
    print(f"LSTM Improvement with {sent}: {imp:.4f}")
    
    results_summary.append(hyb_metrics)

c:\Users\Lenovo\.conda\envs\stable_diff_model\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


--- LSTM_Baseline_AR_Only ---
MAE:  0.3514
RMSE: 0.5008
R2:   -0.0041
------------------------------


c:\Users\Lenovo\.conda\envs\stable_diff_model\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


--- LSTM_Hybrid_insei_indobert_raw ---
MAE:  0.3595
RMSE: 0.5051
R2:   -0.0212
------------------------------
LSTM Improvement with insei_indobert_raw: -0.0042


c:\Users\Lenovo\.conda\envs\stable_diff_model\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


--- LSTM_Hybrid_insei_indolem_raw ---
MAE:  0.3635
RMSE: 0.5084
R2:   -0.0349
------------------------------
LSTM Improvement with insei_indolem_raw: -0.0076


c:\Users\Lenovo\.conda\envs\stable_diff_model\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


--- LSTM_Hybrid_insei_mdeberta_raw ---
MAE:  0.3617
RMSE: 0.5072
R2:   -0.0297
------------------------------
LSTM Improvement with insei_mdeberta_raw: -0.0063


c:\Users\Lenovo\.conda\envs\stable_diff_model\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


--- LSTM_Hybrid_insei_gemini_raw ---
MAE:  0.3633
RMSE: 0.5083
R2:   -0.0342
------------------------------
LSTM Improvement with insei_gemini_raw: -0.0074


LSTM is a Recurrent Neural Network (RNN) designed to capture non-linear patterns and long-term dependencies (memory).
- The Core Logic:At each time step $t$, the LSTM cell takes input $x_t$ (Rate + Sentiment) and the previous hidden state $h_{t-1}$. It computes:
    1. Forget Gate ($f_t$): What to throw away from memory.$$f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)$$
    2. Input Gate ($i_t$): What new information to store.$$i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)$$
    3. Cell State ($C_t$): The long-term memory update.$$C_t = f_t * C_{t-1} + i_t * \tanh(W_C \cdot [h_{t-1}, x_t] + b_C)$$
    4. Output Gate ($o_t$): The prediction for the next step.$$h_t = o_t * \tanh(C_t)$$

The LSTM architecture consists of a single layer with 64 units (relu activation), followed by a Dropout layer (0.1) to prevent overfitting, and a Dense output layer.
- Input Structure: The data is windowed with a LOOKBACK of 12 months.
    - Baseline: Input shape (Batch, 12, 1) containing only Rate history.
    - Hybrid: Input shape (Batch, 12, 2) containing Rate and Sentiment history
- Training: The model is trained for 50 epochs with EarlyStopping to halt training when the loss plateaus.
- Forecasting: A rolling forecast loop is implemented. For each step in the test set, the model predicts $t+1$. Crucially, for the hybrid model, the known sentiment at time $t$ (which corresponds to news from $t-2$ due to the lag) is fed into the model to predict the inflation at $t+1$.

The LSTM results are counter-intuitive:
- Baseline RMSE:0.5009
- Hybrid (IndoBERT) RMSE: 0.5045
- Result: The sentiment index actually increased the error (negative improvement) for IndoBERT and others. (-<1%)

Deep learning models are data-hungry. With only 82 training samples, the added complexity of a second feature (sentiment) likely introduced more variance than signal, causing the model to overfit the noise rather than learn the relationship. This finding suggests that for small macroeconomic datasets, simpler econometric models (VAR) or regularized linear models (Ridge) are superior to deep learning.

# Summary

In [8]:
summary_df = pd.DataFrame(results_summary)
summary_df = summary_df.sort_values("RMSE")
summary_df.to_csv("forecasting_results_raw.csv", index=False)

best_model = summary_df.iloc[0]["Model"]
print(f"Best Performing Model: {best_model}\n")

print("SUMMARY")
summary_df

Best Performing Model: VAR_Baseline_AR_Only

SUMMARY


,Model,MAE,RMSE,R2
0,VAR_Baseline_AR_Only,0.349903,0.500217,-0.001693
10,LSTM_Baseline_AR_Only,0.351350,0.500815,-0.004090
1,VAR_Hybrid_insei_indobert_raw,0.356089,0.504162,-0.017555
11,LSTM_Hybrid_insei_indobert_raw,0.359506,0.505053,-0.021155
3,VAR_Hybrid_insei_mdeberta_raw,0.354518,0.506502,-0.027023
13,LSTM_Hybrid_insei_mdeberta_raw,0.361711,0.507165,-0.029713
14,LSTM_Hybrid_insei_gemini_raw,0.363312,0.508258,-0.034156
12,LSTM_Hybrid_insei_indolem_raw,0.363509,0.508442,-0.034905
2,VAR_Hybrid_insei_indolem_raw,0.359836,0.508591,-0.035512
4,VAR_Hybrid_insei_gemini_raw,0.358160,0.509564,-0.039477


Top performaer was a baseline model which prrof that with raw INSEI Index is no better than without it. That is why lets next try with the lagged index.